In [1]:
import nlgserv
import json
import urllib2
import subprocess
import os
from time import sleep

In [2]:
port = 8080

In [3]:
jython_jar = os.path.join('nlgserv','jython.jar')
server_py = os.path.join('nlgserv','_server.py')
outdir = '.'

In [4]:
def setUpModule(port):
    global nlgserv
    print "Starting up nlgserv..."
    nlgserv = subprocess.Popen([jython_jar,
                                server_py,
                                "localhost", str(port)],
                               stdin=subprocess.PIPE,
                               stdout=open("nlgserv.stdout.log", "w+"),
                               stderr=open("nlgserv.stderr.log", "w+"),
                               preexec_fn=os.setsid)
    sleep(60) # It needs longer now it's loading from the standalone package...
    print "Commencing testing..."
    
def tearDownModule():
    global nlgserv
    print "Shutting down nlgserv..."
    os.killpg(nlgserv.pid, subprocess.signal.SIGTERM)
    nlgserv.wait()

In [5]:
def send_data(json_data, port):
    the_url = "http://localhost:%d/generateSentence"%(port)
    print the_url
    req = urllib2.Request(the_url,
                          data=json_data,
                          headers={"Content-Type":"application/json"})
    return urllib2.urlopen(req).read()


In [6]:
setUpModule(port)

Starting up nlgserv...
Commencing testing...


In [43]:
sentence = {}
sentence['subject'] = 'Yi'
sentence['verb'] = 'leave'

In [44]:
send_data(json.dumps({'sentence':sentence}),port)

http://localhost:8080/generateSentence


'Yi leaves.'

In [46]:
sentence['features'] = {'tense':'past'}
send_data(json.dumps({'sentence':sentence}),port)

http://localhost:8080/generateSentence


'Yi left.'

In [47]:
sentence['features'] = {'tense':'future'}
send_data(json.dumps({'sentence':sentence}),port)

http://localhost:8080/generateSentence


'Yi will leave.'

In [48]:
sentence['features'] = {'tense':'future','passive':'true'}
send_data(json.dumps({'sentence':sentence}),port)

http://localhost:8080/generateSentence


'Will be left by Yi.'

In [50]:
sentence= {}
sentence['subject'] = 'Yi'
sentence['verb'] = 'leave'
sentence['object'] = 'a note'
send_data(json.dumps({'sentence':sentence}),port)

http://localhost:8080/generateSentence


'Yi leaves a note.'

In [51]:
sentence= {}
sentence['subject'] = 'Yi'
sentence['verb'] = 'leave'
sentence['object'] = 'a note'
sentence['features']={'passive':'true','tense':'past'}
send_data(json.dumps({'sentence':sentence}),port)

http://localhost:8080/generateSentence


'A note was left by Yi.'

In [41]:
sentence= {}
sentence['subject'] = 'Yi'
sentence['verb'] = 'leave'
sentence['object'] = {'type':'noun_phrase','head':'note','determiner':'the'}
sentence['features']={'passive':'true','tense':'present'}
send_data(json.dumps({'sentence':sentence}),port)

http://localhost:8080/generateSentence


'The note is left by Yi.'

In [78]:
sentence = {}
sentence["subject"] = "John"
sentence["verb"] = "kick"
sentence["object"] = "Dave"
#sentence['adverb'] = 'happily'

#sentence["features"] = {"tense":"past",'passive':'true','negated':'true'}
#sentence["features"] = {"tense":"past",'passive':'true'}
sentence["features"] = {}

In [81]:
send_data(json.dumps({'sentence':sentence}),port)

http://localhost:8080/generateSentence


'John kicks Dave.'

In [82]:
sentence['subject'] = 'John and Mary'

In [83]:
send_data(json.dumps({'sentence':sentence}),port)

http://localhost:8080/generateSentence


'John and Mary kicks Dave.'

In [86]:
sentence['subject'] = {'head':'John and Mary','features':{'number':'plural'},'type':'noun_phrase'}

In [13]:
tearDownModule()

Shutting down nlgserv...
